# experiment with SAE
https://transformer-circuits.pub/2023/monosemantic-features

In [6]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F

In [8]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [ ]:
# class Dataset(Dataset):
#     def __init__(self):
#         super().__init__()
#         # self.dataframe = train_df

#     def __len__(self):
#         return len(self.dataframe)

#     def __getitem__(self, idx): # preprocessed version
#         row = self.dataframe.iloc[idx]
#         id = row['spectrogram_id']
#         sub_id = row['spectrogram_sub_id']
#         path = f'{PRE_PROCESSED_PATH}/{id}_{sub_id}.pt'
#         data = t.load(path)
#         labels = row[TARGETS].values.astype(np.float64)
#         labels = labels / np.sum(labels)
#         labels_out = t.tensor(labels, dtype=t.float64)
#         return data, labels_out


# ds_len = 10
# dataset = []
# for i in range(ds_len):
#     # call the model we want to apply SAE to, in order to generate training data (aka. activations of MLP)
#     dataset.append(gpt2(input).hook)



In [7]:
class SAE(nn.Module):
    def __init__(self, n_in, n_hidden):
        super(SAE, self).__init__()
        self.b_a = nn.Parameter(t.zeros(n_in))
        self.W_e = nn.Linear(n_in, n_hidden, bias=False)
        self.b_e = nn.Parameter(t.zeros(n_hidden))
        self.W_d = nn.Linear(n_hidden, n_in, bias=False)
        self.b_d = nn.Parameter(t.zeros(n_in))

    def forward(self, act):
        x = act + self.b_a
        x = self.W_e(x) + self.b_e
        x = F.relu(x)
        hidden_activation = x
        x = self.W_d(x) + self.b_d
        x = F.relu(x)
        return x, hidden_activations

model = SAE(n_in=10, n_hidden=20).to(device)

In [ ]:
def train(model, opt, dataset, epochs=100, l1_factor=0.1):
    for epoch in range(epochs):
        for act in dataset:
            logits, hidden_activations = model(act)
            reconstruction_loss = F.mse_loss(logits, act)
            sparsity_loss = hidden_activations.abs().mean()
            loss = reconstruction_loss + l1_factor * sparsity_loss
            opt.zero_grad()
            loss.backward()
            opt.step()

